**Handling of suspected duplicate images**

In [11]:
import pandas as pd

In [12]:
df = pd.read_parquet(r'C:\Users\Noel\Documents\THESIS\Feature Visualization\Dataframes\reapeatability_resnet34.parquet')
# df.tail()

In [13]:
# layers = df.columns.tolist()
# layers = layers[3:]
# layer_dict = {}
# for index in range(len(layers)):
#     layer_dict[index] = layers[index]
# print(layer_dict)

In [14]:
# type(df['conv1'][0])
layers_of_interest = [name for name in df.columns.tolist() if "conv" in name or "fc" in name or "downsample" in name]
print(layers_of_interest)
layer_dict = {}
for index in range(len(layers_of_interest)):
    layer_dict[index] = layers_of_interest[index]
print(layer_dict)

['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.downsample.0', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.3.conv1', 'layer2.3.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.downsample.0', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.5.conv1', 'layer3.5.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.2.conv1', 'layer4.2.conv2', 'fc']
{0: 'conv1', 1: 'layer1.0.conv1', 2: 'layer1.0.conv2', 3: 'layer1.1.conv1', 4: 'layer1.1.conv2', 5: 'layer1.2.conv1', 6: 'layer1.2.conv2', 7: 'layer2.0.conv1', 8: 'layer2.0.conv2', 9: 'layer2.0.downsample.0', 10: 'layer2.1.conv1', 11: 'layer2.1.conv2', 12: 'layer2.2.conv1', 13: 'layer2.2.conv2', 14: 'layer2.3.conv1', 15: 'layer2.3.

In [15]:
df['layer'] = df['layer'].map(lambda x: layer_dict[x])
# df.tail()

In [16]:
# create a new column with which to sort paths
layer = 'layer3.2.conv1'
channel = 78
assert channel <= len(df[layer]), f"Channel Index out of range. {layer} has {len(df[layer])} channels."
column_name = f'activations_{layer}_channel_{channel}'
df[column_name] = df[layer].map(lambda x:x[channel])

In [17]:
df.sort_values(by=column_name, inplace=True, ascending=False)

The immediate following commented out since it cannot work the same way... will delete.

In [18]:
# top_labels =  df['class_label'].head(20).tolist()
# top_predictions =  df['prediction'].head(20).tolist()
# type(top_labels)
# corrects = 0
# for i in range(len(top_labels)):
#     if top_predictions[i] == top_labels[i]:
#         corrects += 1
# percentage = corrects / len(top_labels) * 100
# print(f'Percentage of class labels exciting the channel and being of correct prediction: {percentage}%')

Maybe try the zip method, or enumerate to make the tuple, although this one seems simple and straightforward enough, it does not inform of any errors.

In [19]:
# Get top iamges that activate maximally to pass to grid function
amount = 20
paths = df["path"][:amount].tolist()
layer_names = df["layer"][:amount].tolist()

# Make tuple of (image_path, layer_name)
top_images = [(paths[i], layer_names[i]) for i in range(len(paths))]


In [23]:
import torchvision

from torchvision.transforms import ToTensor
from PIL import ImageDraw, ImageFont, Image
from torchvision.utils import make_grid

# Specify font... maybe do arial.ttf if nothing else works
font = ImageFont.truetype("MOD20.TTF", 35)
position = (15, 180)

images_to_disp = []
# you need a font to change the size of the letters
for image_path, layer_name in list(top_images):
    img_w_legend = Image.open(image_path)
    legend = ImageDraw.Draw(img_w_legend)
    # Define bounding box to place image legend in
    bbox = legend.textbbox(position, layer_name, font=font)
    # bbox = font.getbbox(layer_name)  # newer method, but the syntax should be different... maybe investigate?
    # Design rectangular shape with bbox coordinates
    legend.rectangle(xy=bbox, fill='white')
    # Place the respective text in legend
    legend.text(position, layer_name, font=font, fill='black')
    # Convert to tensor so you can use the 'make_grid' function
    make_tensor = ToTensor()
    # Append to list
    images_to_disp.append(make_tensor(img_w_legend))

# Make the grid
grid = make_grid(images_to_disp, nrow=5, padding=0)

# Pass the grid as PIL Image to display or save
img = torchvision.transforms.ToPILImage()(grid)
img.show()
# img.save(fp=r"")